# Projeto Grupo BT-G3


## **Integrantes do grupo**
- Daniel Barzilai
- Larissa Carvalho
- Maria Luisa Maia
- Pedro Rezende
- Rafael Moritz
- Vitor Oliveira

<center><img src="https://www.inteli.edu.br/wp-content/uploads/2021/08/20172028/marca_1-2.png" width="50%" height="50%"/></center>

<h1 align='center'><b>IA para Marketing: Monitoramento de campanhas utilizando processamento de linguagem natural (PLN)<b></h1>

<center><img src="https://upload.wikimedia.org/wikipedia/commons/c/c2/Btg-logo-blue.svg" width="50%" height="50%"/></center>

<h2 align='center'>O Banco BTG Pactual enfrenta um desafio na área de Marketing em entender as necessidades e demandas dos clientes de maneira fácil e rápida nas redes sociais. A solução proposta para esse problema foi o desenvolvimento de uma Inteligência Artificial utilizando processamento de linguagem natural (PLN), capaz de monitorar as campanhas de marketing, voltadas para o Instagram. O objetivo principal dessa solução é rastrear os dados em tempo real, analisar e interpretar as mensagens e comentários enviados pelos clientes na rede social, a fim de identificar as necessidades e demandas de forma precisa e eficiente.</h2>

---

# Sobre os dados

Esse projeto está utilizando dados coletados e tratados pela equipe de Automation do BTG Pactual, o qual disponibilizou o dataset. Com base nas informações dispostas nesse dataset, realizaremos insights a cerca dos comentários feitos nos posts do Instagram do próprio banco. Vale lembrar que os dados estão anonimizados e resguardados para manter a privacidade e ética com os usuários e com o banco.

# 1. Instalação / Setup

Para o início do projeto, fizemos o desenvolvimento no Google Colab, por isso temos uma célula de conexão com o Google Drive, para poder acessar os dados. Caso seja rodado no Jupyter Notebook, precisará do dataset baixado.

In [17]:
#Conectar com o Google Drive

from google.colab import drive
drive.mount('/content/drive')

#Conectando o ambiente ao Google Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Aqui nós fazemos as importações para tratamento dos dados, pré-processamento dos dados e modelamento do Bag of Words. 

## pips

In [18]:
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
!pip install wordcloud

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Imports

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import re

from sklearn.feature_extraction.text import CountVectorizer
import ast
from keras.preprocessing.text import Tokenizer
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential 
from keras.layers import Dense, Embedding, GlobalMaxPooling1D, Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
from keras.utils import pad_sequences

import nltk
import spacy
import gensim
import pickle
from scipy.spatial.distance import cosine
from gensim.models import KeyedVectors

O desenvolvimento da rede neural seguiu o tutorial abaixo:
https://medium.com/@jvsavietto6/classificando-texto-com-redes-neurais-artificiais-150ef448b13d

# 2. Entendimento e Tratamento dos Dados

Rodando o dataset, para analisar seu conteúdo:

In [24]:
df = pd.read_csv('/content/drive/MyDrive/Módulo 6/Sprint 4 - Projeto/nova_base_tratada')
df

,autor,sentimento,texto_tratado
0,winthegame_of,1,"['alvarez', 'marsal', 'estar', 'conosco', 'spo..."
1,marta_bego,1,"['btgpactual', 'with', 'make_repost', 'entenda..."
2,lmviapiana,2,"['minuto', 'touro', 'ouro']"
3,ricktolledo,2,"['querer', 'saber', 'banking', 'administrar']"
4,donadabolsaoficial,0,"['início', 'guerra', 'rússia', 'ucrânia', 'vel..."
...,...,...,...
8035,perspectiveinvestimentos,2,"['excelente', 'explicação']"
8036,eduardocolares,2,"['atendar', 'telefone', 'amor', 'deus']"
8037,danielucm,2,"['grande', 'fiis', 'mercado', 'não', 'selecion..."
8038,amgcapitalinvest,1,"['erro', 'financeiro', 'eliminar', 'parar', 'o..."


In [25]:
df.columns

Index(['autor', 'sentimento', 'texto_tratado'], dtype='object')

In [26]:
df['texto_tratado']

0       ['alvarez', 'marsal', 'estar', 'conosco', 'spo...
1       ['btgpactual', 'with', 'make_repost', 'entenda...
2                             ['minuto', 'touro', 'ouro']
3           ['querer', 'saber', 'banking', 'administrar']
4       ['início', 'guerra', 'rússia', 'ucrânia', 'vel...
                              ...                        
8035                          ['excelente', 'explicação']
8036              ['atendar', 'telefone', 'amor', 'deus']
8037    ['grande', 'fiis', 'mercado', 'não', 'selecion...
8038    ['erro', 'financeiro', 'eliminar', 'parar', 'o...
8039    ['morning', 'call', 'não', 'aparecer', 'spotif...
Name: texto_tratado, Length: 8040, dtype: object

In [27]:
# Supondo que seu DataFrame seja chamado de df e a coluna seja 'texto_tratado'
df['texto_tratado'] = df['texto_tratado'].str.replace("'", "")
df['texto_tratado']

0       [alvarez, marsal, estar, conosco, sportainmet,...
1       [btgpactual, with, make_repost, entendar, impa...
2                                   [minuto, touro, ouro]
3                   [querer, saber, banking, administrar]
4       [início, guerra, rússia, ucrânia, velho, apare...
                              ...                        
8035                              [excelente, explicação]
8036                      [atendar, telefone, amor, deus]
8037    [grande, fiis, mercado, não, selecionar, princ...
8038    [erro, financeiro, eliminar, parar, ostentar, ...
8039    [morning, call, não, aparecer, spotify, atuali...
Name: texto_tratado, Length: 8040, dtype: object

# 10. Word2Vec com CBOW

## Estruturação

In [28]:
df

,autor,sentimento,texto_tratado
0,winthegame_of,1,"[alvarez, marsal, estar, conosco, sportainmet,..."
1,marta_bego,1,"[btgpactual, with, make_repost, entendar, impa..."
2,lmviapiana,2,"[minuto, touro, ouro]"
3,ricktolledo,2,"[querer, saber, banking, administrar]"
4,donadabolsaoficial,0,"[início, guerra, rússia, ucrânia, velho, apare..."
...,...,...,...
8035,perspectiveinvestimentos,2,"[excelente, explicação]"
8036,eduardocolares,2,"[atendar, telefone, amor, deus]"
8037,danielucm,2,"[grande, fiis, mercado, não, selecionar, princ..."
8038,amgcapitalinvest,1,"[erro, financeiro, eliminar, parar, ostentar, ..."


In [29]:
cbow = '/content/drive/MyDrive/Módulo 6/Semana 5/cbow_s50/cbow_s50.txt'

In [30]:
model_cbow = KeyedVectors.load_word2vec_format(cbow)

## Teste isolado

In [31]:
# Testando o word2vec
wordvec_test = model_cbow['projeto']

wordvec_test

array([-0.074174, -0.152088,  0.086627, -0.224567,  0.362562,  0.130683,
       -0.089179, -0.086973,  0.309501,  0.004112, -0.308202,  0.351789,
       -0.477863,  0.050276,  0.213283,  0.159895, -0.285545, -0.08832 ,
       -0.015449,  0.014816, -0.613861,  0.502556,  0.021688,  0.369492,
        0.280691,  0.016868,  0.105584, -0.180754, -0.078456,  0.148032,
        0.36293 , -0.011634,  0.412191, -0.009049,  0.010404,  0.131242,
       -0.032483, -0.133067, -0.063802,  0.434015, -0.214768, -0.072132,
        0.045601, -0.368866,  0.502808,  0.048293, -0.254894,  0.142581,
       -0.075066,  0.015646], dtype=float32)

## Definição de função

In [32]:
def create_sentence_vector(model, df):
    sentence_table = []
    for sentence in df['texto_tratado']:
        word_vectors = [model[word] for word in sentence if word in model]
        if len(word_vectors) > 0:
            sentence_vector = sum(word_vectors) / len(word_vectors)
        else:
            sentence_vector = [None] * 100  # Cria uma lista de 100 elementos None
        sentence_table.append((sentence, *sentence_vector[:50]))  # Adiciona apenas os primeiros 50 elementos do vetor

    column_labels = ['Frase']
    for i in range(50):
        column_labels.append(f'Vetor{i+1}')
    df_vec = pd.DataFrame(sentence_table, columns=column_labels)

    df["sentimentoNumerico"] = df["sentimento"].replace({'NEGATIVE': -1, 'POSITIVE': 1, 'NEUTRAL': 0})

    # Definir o índice do DataFrame df_vec como o mesmo índice de df_processada['sentimentoNumerico']
    df_vec.set_index(df["sentimentoNumerico"].index, inplace=True)

    df_vec['sentimento'] = df["sentimentoNumerico"]
    df_vec = df_vec.dropna()

    return df_vec

## Teste de funções

In [33]:
df_vec = create_sentence_vector(model_cbow, df)
df_vec

,Frase,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,Vetor6,Vetor7,Vetor8,Vetor9,...,Vetor42,Vetor43,Vetor44,Vetor45,Vetor46,Vetor47,Vetor48,Vetor49,Vetor50,sentimento
0,"[alvarez, marsal, estar, conosco, sportainmet,...",0.214823,-0.123036,0.220707,-0.079739,-0.019442,0.202300,0.073742,0.042477,0.057268,...,0.012086,-0.096122,0.148153,0.092629,-0.027504,0.149860,-0.029617,0.032065,0.190372,1
1,"[btgpactual, with, make_repost, entendar, impa...",0.220860,-0.123713,0.205813,-0.063700,-0.010433,0.195687,0.072226,0.026175,0.058045,...,0.006447,-0.076174,0.157446,0.085306,-0.001963,0.156306,-0.019182,0.035594,0.192635,1
2,"[minuto, touro, ouro]",0.265227,-0.068285,0.152235,-0.044329,-0.102729,0.141353,0.092800,0.113174,0.015783,...,0.078032,-0.202677,0.155750,0.062291,0.007038,0.134573,0.014635,0.034189,0.345674,2
3,"[querer, saber, banking, administrar]",0.189241,-0.169407,0.281181,-0.093912,-0.001968,0.181633,0.080923,0.055674,0.103454,...,0.092738,-0.160601,0.179499,0.091803,-0.063762,0.171190,-0.043618,0.030563,0.196250,2
4,"[início, guerra, rússia, ucrânia, velho, apare...",0.219838,-0.138311,0.229412,-0.057420,-0.014510,0.212875,0.058865,0.068096,0.060398,...,0.040836,-0.112662,0.170401,0.094290,-0.046000,0.168181,-0.018029,0.027667,0.180134,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8035,"[excelente, explicação]",0.190917,-0.133475,0.241675,-0.053180,0.067256,0.201138,0.034109,-0.078718,-0.066131,...,-0.082151,0.016113,0.154861,0.068700,-0.004302,0.079717,-0.028388,-0.017448,0.188785,2
8036,"[atendar, telefone, amor, deus]",0.188641,-0.119377,0.199339,-0.105448,0.023176,0.178837,0.069476,-0.004494,0.034710,...,0.034035,-0.126673,0.165176,0.080313,-0.024160,0.118848,-0.003502,0.087053,0.215656,2
8037,"[grande, fiis, mercado, não, selecionar, princ...",0.219437,-0.141124,0.217854,-0.065569,-0.020409,0.205403,0.061334,0.041665,0.063050,...,0.031438,-0.099000,0.176588,0.092546,-0.046104,0.145734,-0.019857,0.029422,0.183504,2
8038,"[erro, financeiro, eliminar, parar, ostentar, ...",0.218153,-0.146420,0.245234,-0.056113,-0.036187,0.224789,0.074320,0.045111,0.040841,...,0.035260,-0.126793,0.171115,0.095479,-0.033473,0.158747,-0.006721,0.029326,0.198914,1


In [34]:
#df_vec.to_csv('Word2Vec_Cbow_modelo_treinado',encoding='utf-8', index=False, header=True)

# 11. Naive Bayes + Word2Vec com CBOW

In [35]:
label = preprocessing.LabelEncoder()

In [36]:
label.fit(df_vec['sentimento'])
df_vec['sentimento'] = label.transform(df_vec['sentimento'])

In [37]:
df_vec = df_vec.dropna()
df_vec

,Frase,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,Vetor6,Vetor7,Vetor8,Vetor9,...,Vetor42,Vetor43,Vetor44,Vetor45,Vetor46,Vetor47,Vetor48,Vetor49,Vetor50,sentimento
0,"[alvarez, marsal, estar, conosco, sportainmet,...",0.214823,-0.123036,0.220707,-0.079739,-0.019442,0.202300,0.073742,0.042477,0.057268,...,0.012086,-0.096122,0.148153,0.092629,-0.027504,0.149860,-0.029617,0.032065,0.190372,1
1,"[btgpactual, with, make_repost, entendar, impa...",0.220860,-0.123713,0.205813,-0.063700,-0.010433,0.195687,0.072226,0.026175,0.058045,...,0.006447,-0.076174,0.157446,0.085306,-0.001963,0.156306,-0.019182,0.035594,0.192635,1
2,"[minuto, touro, ouro]",0.265227,-0.068285,0.152235,-0.044329,-0.102729,0.141353,0.092800,0.113174,0.015783,...,0.078032,-0.202677,0.155750,0.062291,0.007038,0.134573,0.014635,0.034189,0.345674,2
3,"[querer, saber, banking, administrar]",0.189241,-0.169407,0.281181,-0.093912,-0.001968,0.181633,0.080923,0.055674,0.103454,...,0.092738,-0.160601,0.179499,0.091803,-0.063762,0.171190,-0.043618,0.030563,0.196250,2
4,"[início, guerra, rússia, ucrânia, velho, apare...",0.219838,-0.138311,0.229412,-0.057420,-0.014510,0.212875,0.058865,0.068096,0.060398,...,0.040836,-0.112662,0.170401,0.094290,-0.046000,0.168181,-0.018029,0.027667,0.180134,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8035,"[excelente, explicação]",0.190917,-0.133475,0.241675,-0.053180,0.067256,0.201138,0.034109,-0.078718,-0.066131,...,-0.082151,0.016113,0.154861,0.068700,-0.004302,0.079717,-0.028388,-0.017448,0.188785,2
8036,"[atendar, telefone, amor, deus]",0.188641,-0.119377,0.199339,-0.105448,0.023176,0.178837,0.069476,-0.004494,0.034710,...,0.034035,-0.126673,0.165176,0.080313,-0.024160,0.118848,-0.003502,0.087053,0.215656,2
8037,"[grande, fiis, mercado, não, selecionar, princ...",0.219437,-0.141124,0.217854,-0.065569,-0.020409,0.205403,0.061334,0.041665,0.063050,...,0.031438,-0.099000,0.176588,0.092546,-0.046104,0.145734,-0.019857,0.029422,0.183504,2
8038,"[erro, financeiro, eliminar, parar, ostentar, ...",0.218153,-0.146420,0.245234,-0.056113,-0.036187,0.224789,0.074320,0.045111,0.040841,...,0.035260,-0.126793,0.171115,0.095479,-0.033473,0.158747,-0.006721,0.029326,0.198914,1


## Separando Treino e Teste

In [38]:
target = df_vec['sentimento']

In [39]:
feature = df_vec.iloc[:,1:50]

In [40]:
feature

,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,Vetor6,Vetor7,Vetor8,Vetor9,Vetor10,...,Vetor40,Vetor41,Vetor42,Vetor43,Vetor44,Vetor45,Vetor46,Vetor47,Vetor48,Vetor49
0,0.214823,-0.123036,0.220707,-0.079739,-0.019442,0.202300,0.073742,0.042477,0.057268,0.168187,...,0.076954,-0.281536,0.012086,-0.096122,0.148153,0.092629,-0.027504,0.149860,-0.029617,0.032065
1,0.220860,-0.123713,0.205813,-0.063700,-0.010433,0.195687,0.072226,0.026175,0.058045,0.163238,...,0.086821,-0.305640,0.006447,-0.076174,0.157446,0.085306,-0.001963,0.156306,-0.019182,0.035594
2,0.265227,-0.068285,0.152235,-0.044329,-0.102729,0.141353,0.092800,0.113174,0.015783,0.202198,...,-0.008447,-0.193025,0.078032,-0.202677,0.155750,0.062291,0.007038,0.134573,0.014635,0.034189
3,0.189241,-0.169407,0.281181,-0.093912,-0.001968,0.181633,0.080923,0.055674,0.103454,0.144342,...,0.060960,-0.254610,0.092738,-0.160601,0.179499,0.091803,-0.063762,0.171190,-0.043618,0.030563
4,0.219838,-0.138311,0.229412,-0.057420,-0.014510,0.212875,0.058865,0.068096,0.060398,0.156307,...,0.067551,-0.269136,0.040836,-0.112662,0.170401,0.094290,-0.046000,0.168181,-0.018029,0.027667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8035,0.190917,-0.133475,0.241675,-0.053180,0.067256,0.201138,0.034109,-0.078718,-0.066131,0.187608,...,0.014565,-0.321192,-0.082151,0.016113,0.154861,0.068700,-0.004302,0.079717,-0.028388,-0.017448
8036,0.188641,-0.119377,0.199339,-0.105448,0.023176,0.178837,0.069476,-0.004494,0.034710,0.150081,...,0.071114,-0.194663,0.034035,-0.126673,0.165176,0.080313,-0.024160,0.118848,-0.003502,0.087053
8037,0.219437,-0.141124,0.217854,-0.065569,-0.020409,0.205403,0.061334,0.041665,0.063050,0.167985,...,0.077558,-0.300234,0.031438,-0.099000,0.176588,0.092546,-0.046104,0.145734,-0.019857,0.029422
8038,0.218153,-0.146420,0.245234,-0.056113,-0.036187,0.224789,0.074320,0.045111,0.040841,0.150971,...,0.060317,-0.270532,0.035260,-0.126793,0.171115,0.095479,-0.033473,0.158747,-0.006721,0.029326


In [41]:
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.2, random_state=42)

## Avaliação do modelo 

In [42]:
clf = GaussianNB()

clf = clf.fit(X_train,y_train.values.ravel())

Y_pred = clf.predict(X_test)

print(classification_report(y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.25      0.73      0.37       360
           1       0.51      0.30      0.38       597
           2       0.32      0.10      0.15       651

    accuracy                           0.31      1608
   macro avg       0.36      0.38      0.30      1608
weighted avg       0.38      0.31      0.28      1608



In [43]:
acc_score = accuracy_score(y_test, Y_pred)
format_output = "{:.2%}".format(acc_score)
print("Precisão final de :",format_output) 

Precisão final de : 31.47%


# 12. Word2Vec com embedding layer

## Definição de função

In [44]:
from gensim.models import Word2Vec

# Função que treina o modelo Word2Vec no corpus do dataframe
def train_word2vec(df, column_name):
    # Obtém as frases tokenizadas
    sentences = df[column_name].tolist()
    
    # Treina o modelo Word2Vec
    model = Word2Vec(sentences, min_count=1)
    
    return model

In [45]:
# Função que define os vetores para cada palavra do vocabulario
def get_word_vectors(model, sentence):
    vectors = []
    for word in sentence:
        if word in model.wv:
            vectors.append(model.wv[word]) # Append na lista de vetores
    if vectors:
        return np.sum(vectors, axis=0)/len(sentence) # Soma dos vetores para cada frase
    else:
        return np.zeros(model.vector_size)

# Criação do dataframe de vetores para cada frase
def create_word2vec_dataframe(df, column_name, model):
    sentences = df[column_name].tolist()
    vectors = [get_word_vectors(model, sentence) for sentence in sentences] # Itera para cada frase um vetor
    # Criação do dataframe
    df_vectors = pd.DataFrame(vectors, columns=[f"Vetor{i}" for i in range(model.vector_size)])
    df_word2vec = pd.concat([df, df_vectors], axis=1)
    return df_word2vec

## Teste de funções

In [46]:
model = train_word2vec(df, 'texto_tratado')

In [47]:
df_word2vec = create_word2vec_dataframe(df,'texto_tratado', model)
df_word2vec

,autor,sentimento,texto_tratado,sentimentoNumerico,Vetor0,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,...,Vetor90,Vetor91,Vetor92,Vetor93,Vetor94,Vetor95,Vetor96,Vetor97,Vetor98,Vetor99
0,winthegame_of,1,"[alvarez, marsal, estar, conosco, sportainmet,...",1,0.297414,0.006346,0.135038,0.061838,-0.472301,0.233506,...,-0.529049,-0.093984,-0.277712,0.600598,-0.224913,0.077593,-0.300101,0.015710,-0.027120,-0.124745
1,marta_bego,1,"[btgpactual, with, make_repost, entendar, impa...",1,0.281173,0.013540,0.112198,0.077889,-0.447345,0.228404,...,-0.493123,-0.097028,-0.298142,0.570242,-0.218310,0.051990,-0.291846,0.035265,-0.034986,-0.114778
2,lmviapiana,2,"[minuto, touro, ouro]",2,0.288293,0.077489,0.083979,-0.029461,-0.449480,0.319432,...,-0.526300,-0.094131,-0.340655,0.586246,-0.188992,0.181223,-0.175162,-0.040045,0.063862,-0.208368
3,ricktolledo,2,"[querer, saber, banking, administrar]",2,0.268780,0.011891,0.103685,0.073322,-0.418163,0.195450,...,-0.505033,-0.096492,-0.281548,0.480524,-0.226503,0.106688,-0.250742,-0.008421,-0.057605,-0.068400
4,donadabolsaoficial,0,"[início, guerra, rússia, ucrânia, velho, apare...",0,0.273418,0.049957,0.097502,0.070372,-0.475558,0.255420,...,-0.529097,-0.081015,-0.279064,0.594373,-0.248605,0.046594,-0.286832,0.044750,-0.021001,-0.106887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8035,perspectiveinvestimentos,2,"[excelente, explicação]",2,0.233112,0.151018,0.135960,0.022766,-0.479874,0.150638,...,-0.456086,-0.174277,-0.319233,0.522279,-0.304542,-0.031399,-0.251650,0.073567,-0.034155,-0.137709
8036,eduardocolares,2,"[atendar, telefone, amor, deus]",2,0.307020,0.050195,0.037812,0.063392,-0.421072,0.260814,...,-0.526184,-0.092594,-0.331005,0.592767,-0.261083,0.003219,-0.327767,0.041722,-0.047595,-0.152317
8037,danielucm,2,"[grande, fiis, mercado, não, selecionar, princ...",2,0.284146,0.047818,0.108436,0.064484,-0.500483,0.272070,...,-0.501176,-0.081624,-0.290470,0.594903,-0.255342,0.034344,-0.309130,0.058736,-0.004545,-0.090214
8038,amgcapitalinvest,1,"[erro, financeiro, eliminar, parar, ostentar, ...",1,0.281530,0.070156,0.116428,0.075205,-0.506874,0.290723,...,-0.524735,-0.087025,-0.276499,0.614986,-0.221881,0.053634,-0.317234,0.051381,-0.020959,-0.114861


In [48]:
df_word2vec = df_word2vec.drop(columns=['autor', 'sentimento'])
df_word2vec

,texto_tratado,sentimentoNumerico,Vetor0,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,Vetor6,Vetor7,...,Vetor90,Vetor91,Vetor92,Vetor93,Vetor94,Vetor95,Vetor96,Vetor97,Vetor98,Vetor99
0,"[alvarez, marsal, estar, conosco, sportainmet,...",1,0.297414,0.006346,0.135038,0.061838,-0.472301,0.233506,0.117101,-0.425061,...,-0.529049,-0.093984,-0.277712,0.600598,-0.224913,0.077593,-0.300101,0.015710,-0.027120,-0.124745
1,"[btgpactual, with, make_repost, entendar, impa...",1,0.281173,0.013540,0.112198,0.077889,-0.447345,0.228404,0.106523,-0.412592,...,-0.493123,-0.097028,-0.298142,0.570242,-0.218310,0.051990,-0.291846,0.035265,-0.034986,-0.114778
2,"[minuto, touro, ouro]",2,0.288293,0.077489,0.083979,-0.029461,-0.449480,0.319432,0.230896,-0.553132,...,-0.526300,-0.094131,-0.340655,0.586246,-0.188992,0.181223,-0.175162,-0.040045,0.063862,-0.208368
3,"[querer, saber, banking, administrar]",2,0.268780,0.011891,0.103685,0.073322,-0.418163,0.195450,0.059639,-0.359379,...,-0.505033,-0.096492,-0.281548,0.480524,-0.226503,0.106688,-0.250742,-0.008421,-0.057605,-0.068400
4,"[início, guerra, rússia, ucrânia, velho, apare...",0,0.273418,0.049957,0.097502,0.070372,-0.475558,0.255420,0.138153,-0.459308,...,-0.529097,-0.081015,-0.279064,0.594373,-0.248605,0.046594,-0.286832,0.044750,-0.021001,-0.106887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8035,"[excelente, explicação]",2,0.233112,0.151018,0.135960,0.022766,-0.479874,0.150638,0.130819,-0.328104,...,-0.456086,-0.174277,-0.319233,0.522279,-0.304542,-0.031399,-0.251650,0.073567,-0.034155,-0.137709
8036,"[atendar, telefone, amor, deus]",2,0.307020,0.050195,0.037812,0.063392,-0.421072,0.260814,0.164793,-0.466451,...,-0.526184,-0.092594,-0.331005,0.592767,-0.261083,0.003219,-0.327767,0.041722,-0.047595,-0.152317
8037,"[grande, fiis, mercado, não, selecionar, princ...",2,0.284146,0.047818,0.108436,0.064484,-0.500483,0.272070,0.147573,-0.475484,...,-0.501176,-0.081624,-0.290470,0.594903,-0.255342,0.034344,-0.309130,0.058736,-0.004545,-0.090214
8038,"[erro, financeiro, eliminar, parar, ostentar, ...",1,0.281530,0.070156,0.116428,0.075205,-0.506874,0.290723,0.140936,-0.486017,...,-0.524735,-0.087025,-0.276499,0.614986,-0.221881,0.053634,-0.317234,0.051381,-0.020959,-0.114861


# 13. Naive Bayes + Word2Vec com embedding layer

In [49]:
df_word2vec = df_word2vec.dropna()
df_word2vec

,texto_tratado,sentimentoNumerico,Vetor0,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,Vetor6,Vetor7,...,Vetor90,Vetor91,Vetor92,Vetor93,Vetor94,Vetor95,Vetor96,Vetor97,Vetor98,Vetor99
0,"[alvarez, marsal, estar, conosco, sportainmet,...",1,0.297414,0.006346,0.135038,0.061838,-0.472301,0.233506,0.117101,-0.425061,...,-0.529049,-0.093984,-0.277712,0.600598,-0.224913,0.077593,-0.300101,0.015710,-0.027120,-0.124745
1,"[btgpactual, with, make_repost, entendar, impa...",1,0.281173,0.013540,0.112198,0.077889,-0.447345,0.228404,0.106523,-0.412592,...,-0.493123,-0.097028,-0.298142,0.570242,-0.218310,0.051990,-0.291846,0.035265,-0.034986,-0.114778
2,"[minuto, touro, ouro]",2,0.288293,0.077489,0.083979,-0.029461,-0.449480,0.319432,0.230896,-0.553132,...,-0.526300,-0.094131,-0.340655,0.586246,-0.188992,0.181223,-0.175162,-0.040045,0.063862,-0.208368
3,"[querer, saber, banking, administrar]",2,0.268780,0.011891,0.103685,0.073322,-0.418163,0.195450,0.059639,-0.359379,...,-0.505033,-0.096492,-0.281548,0.480524,-0.226503,0.106688,-0.250742,-0.008421,-0.057605,-0.068400
4,"[início, guerra, rússia, ucrânia, velho, apare...",0,0.273418,0.049957,0.097502,0.070372,-0.475558,0.255420,0.138153,-0.459308,...,-0.529097,-0.081015,-0.279064,0.594373,-0.248605,0.046594,-0.286832,0.044750,-0.021001,-0.106887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8035,"[excelente, explicação]",2,0.233112,0.151018,0.135960,0.022766,-0.479874,0.150638,0.130819,-0.328104,...,-0.456086,-0.174277,-0.319233,0.522279,-0.304542,-0.031399,-0.251650,0.073567,-0.034155,-0.137709
8036,"[atendar, telefone, amor, deus]",2,0.307020,0.050195,0.037812,0.063392,-0.421072,0.260814,0.164793,-0.466451,...,-0.526184,-0.092594,-0.331005,0.592767,-0.261083,0.003219,-0.327767,0.041722,-0.047595,-0.152317
8037,"[grande, fiis, mercado, não, selecionar, princ...",2,0.284146,0.047818,0.108436,0.064484,-0.500483,0.272070,0.147573,-0.475484,...,-0.501176,-0.081624,-0.290470,0.594903,-0.255342,0.034344,-0.309130,0.058736,-0.004545,-0.090214
8038,"[erro, financeiro, eliminar, parar, ostentar, ...",1,0.281530,0.070156,0.116428,0.075205,-0.506874,0.290723,0.140936,-0.486017,...,-0.524735,-0.087025,-0.276499,0.614986,-0.221881,0.053634,-0.317234,0.051381,-0.020959,-0.114861


## Separando Treino e Teste

In [50]:
target = df_word2vec['sentimentoNumerico']

In [51]:
feature = df_word2vec.iloc[:,2:102]

In [52]:
feature

,Vetor0,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,Vetor6,Vetor7,Vetor8,Vetor9,...,Vetor90,Vetor91,Vetor92,Vetor93,Vetor94,Vetor95,Vetor96,Vetor97,Vetor98,Vetor99
0,0.297414,0.006346,0.135038,0.061838,-0.472301,0.233506,0.117101,-0.425061,-0.017413,-0.068499,...,-0.529049,-0.093984,-0.277712,0.600598,-0.224913,0.077593,-0.300101,0.015710,-0.027120,-0.124745
1,0.281173,0.013540,0.112198,0.077889,-0.447345,0.228404,0.106523,-0.412592,-0.022186,-0.046436,...,-0.493123,-0.097028,-0.298142,0.570242,-0.218310,0.051990,-0.291846,0.035265,-0.034986,-0.114778
2,0.288293,0.077489,0.083979,-0.029461,-0.449480,0.319432,0.230896,-0.553132,-0.133807,0.019880,...,-0.526300,-0.094131,-0.340655,0.586246,-0.188992,0.181223,-0.175162,-0.040045,0.063862,-0.208368
3,0.268780,0.011891,0.103685,0.073322,-0.418163,0.195450,0.059639,-0.359379,0.011575,-0.039374,...,-0.505033,-0.096492,-0.281548,0.480524,-0.226503,0.106688,-0.250742,-0.008421,-0.057605,-0.068400
4,0.273418,0.049957,0.097502,0.070372,-0.475558,0.255420,0.138153,-0.459308,-0.012264,-0.010020,...,-0.529097,-0.081015,-0.279064,0.594373,-0.248605,0.046594,-0.286832,0.044750,-0.021001,-0.106887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8035,0.233112,0.151018,0.135960,0.022766,-0.479874,0.150638,0.130819,-0.328104,-0.006419,-0.057633,...,-0.456086,-0.174277,-0.319233,0.522279,-0.304542,-0.031399,-0.251650,0.073567,-0.034155,-0.137709
8036,0.307020,0.050195,0.037812,0.063392,-0.421072,0.260814,0.164793,-0.466451,-0.000165,-0.018398,...,-0.526184,-0.092594,-0.331005,0.592767,-0.261083,0.003219,-0.327767,0.041722,-0.047595,-0.152317
8037,0.284146,0.047818,0.108436,0.064484,-0.500483,0.272070,0.147573,-0.475484,-0.013853,0.010120,...,-0.501176,-0.081624,-0.290470,0.594903,-0.255342,0.034344,-0.309130,0.058736,-0.004545,-0.090214
8038,0.281530,0.070156,0.116428,0.075205,-0.506874,0.290723,0.140936,-0.486017,-0.012482,-0.011813,...,-0.524735,-0.087025,-0.276499,0.614986,-0.221881,0.053634,-0.317234,0.051381,-0.020959,-0.114861


In [53]:
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.2, random_state=42)

## Avaliação do modelo 

In [54]:
clf = GaussianNB()

clf = clf.fit(X_train,y_train.values.ravel())

Y_pred = clf.predict(X_test)

print(classification_report(y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.25      0.77      0.37       360
           1       0.59      0.28      0.38       597
           2       0.42      0.14      0.21       651

    accuracy                           0.33      1608
   macro avg       0.42      0.39      0.32      1608
weighted avg       0.45      0.33      0.31      1608



In [55]:
acc_score = accuracy_score(y_test, Y_pred)
format_output = "{:.2%}".format(acc_score)
print("Precisão final de :",format_output) 

Precisão final de : 33.08%


# Rede Neural - Sequência de palavras

## Teste isolado

In [56]:
! pip install pad_sequences

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pad_sequences: filename=pad_sequences-0.6.1-py3-none-any.whl size=10199 sha256=59d015a46820c97233ccce7642abcef4f06bc672304d2cfc4eed8ca88aaa1548
  Stored in directory: /root/.cache/pip/wheels/48/9d/22/0a6305b87a9cc46ccc032060a041c3b59f39ac462f7358997e
Successfully built pad_sequences


In [57]:
from keras.preprocessing.text import Tokenizer
#from keras_preprocessing.sequence import pad_sequences
from keras.utils import pad_sequences

citacao = [
    "Estudo no Inteli e escolhi o curso de Sistemas de Informação",
    "Estamos fazendo um projeto para o BTG",
    "Estamos no segundo ano de graduação",
    "Somos do grupo BTG3"
]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(citacao)
citacao_token = tokenizer.texts_to_sequences(citacao)

max_length = max([len(z) for z in citacao_token]) + 1
citacao_pad = pad_sequences(citacao_token, maxlen=max_length, padding="post")

## Construção da rede neural + resultados - base tratada

A rede neural abaixo foi desenvolvida com o tutorial mencionado no ínicio do notebook, que, nesse caso o dataframe utilizado é um arquivo com a base que já passou pelo pré processamento. 

### Dataset já treinado

In [62]:
teste_rede_neural = pd.read_csv("/content/drive/MyDrive/Módulo 6/Sprint 4 - Projeto/nova_base_tratada")

In [63]:
teste_rede_neural

,autor,sentimento,texto_tratado
0,winthegame_of,1,"['alvarez', 'marsal', 'estar', 'conosco', 'spo..."
1,marta_bego,1,"['btgpactual', 'with', 'make_repost', 'entenda..."
2,lmviapiana,2,"['minuto', 'touro', 'ouro']"
3,ricktolledo,2,"['querer', 'saber', 'banking', 'administrar']"
4,donadabolsaoficial,0,"['início', 'guerra', 'rússia', 'ucrânia', 'vel..."
...,...,...,...
8035,perspectiveinvestimentos,2,"['excelente', 'explicação']"
8036,eduardocolares,2,"['atendar', 'telefone', 'amor', 'deus']"
8037,danielucm,2,"['grande', 'fiis', 'mercado', 'não', 'selecion..."
8038,amgcapitalinvest,1,"['erro', 'financeiro', 'eliminar', 'parar', 'o..."


### Separação de treino e teste

In [64]:
x, y = teste_rede_neural["texto_tratado"], teste_rede_neural["sentimento"]

labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)

words = ["o", "ao", 'aos', 'os', 'a', 'as', 'e', 'um', 'uma', 
        'ele', 'ela', 'eles', 'elas', 'do', 'da', 'dos', 'das', 
        'de', 'no', 'na', 'nos', 'nas', 'pelo', 'pela', 'pelos', 
        'pelas', 'num', 'numa', 'nuns', 'numas', 'dum', 'duma', 
        'duns', 'dumas']

x_filter = []

for title in x:
  for word in words:
    title = title.replace(word, '')
  x_filter.append(title)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_filter)

vocab = len(tokenizer.word_docs) + 1

x_filter = tokenizer.texts_to_sequences(x_filter)

max_length = max([len(z) for z in x_filter])
x_filter = pad_sequences(x_filter, maxlen=max_length, padding='post')

x_train, x_test, y_train, y_test = train_test_split(x_filter, y, test_size=0.33)

print("Tamanho de x:", len(x_filter))
print("Tamanho de y:", len(y))


Tamanho de x: 8040
Tamanho de y: 8040


### Criação do modelo

In [65]:
model = Sequential()
model.add(Embedding(input_dim=vocab, output_dim=80, input_length=max_length, trainable = True))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.3))
model.add(Dense(units = 7, activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

mc = ModelCheckpoint('weight.best.hdf5', monitor='val_acc', save_best_only=True, mode='max')

model.fit(x_train, y_train, validation_data = (x_test, y_test), batch_size = 32, epochs = 5, callbacks = [mc])

print(model.evaluate(x_test, y_test))

Epoch 1/5
168/169 [============================>.] - ETA: 0s - loss: 1.4460 - accuracy: 0.4490

169/169 [==============================] - 7s 32ms/step - loss: 1.4454 - accuracy: 0.4491 - val_loss: 1.1021 - val_accuracy: 0.4985
Epoch 2/5
169/169 [==============================] - ETA: 0s - loss: 1.0107 - accuracy: 0.5423

169/169 [==============================] - 8s 45ms/step - loss: 1.0107 - accuracy: 0.5423 - val_loss: 0.9521 - val_accuracy: 0.5671
Epoch 3/5
167/169 [============================>.] - ETA: 0s - loss: 0.8829 - accuracy: 0.6235

169/169 [==============================] - 4s 22ms/step - loss: 0.8823 - accuracy: 0.6242 - val_loss: 0.8619 - val_accuracy: 0.6168
Epoch 4/5
166/169 [============================>.] - ETA: 0s - loss: 0.7804 - accuracy: 0.6930

169/169 [==============================] - 3s 17ms/step - loss: 0.7799 - accuracy: 0.6931 - val_loss: 0.8064 - val_accuracy: 0.6556
Epoch 5/5
167/169 [============================>.] - ETA: 0s - loss: 0.6905 - accuracy: 0.7500

83/83 [==============================] - 0s 3ms/step - loss: 0.7691 - accuracy: 0.6669
[0.7690526247024536, 0.6669178605079651]


In [66]:
# Avaliação do modelo
loss, accuracy = model.evaluate(x_test, y_test)
print('Loss:', loss)
print('Accuracy:', accuracy)

83/83 [==============================] - 0s 3ms/step - loss: 0.7691 - accuracy: 0.6669
Loss: 0.7690526247024536
Accuracy: 0.6669178605079651


### Exportação com a biblioteca pickle

In [67]:
with open('redeNeural_base.pkl', 'wb') as arquivo:
    pickle.dump(model, arquivo)
with open('redeNeural_base.pkl', 'rb') as arquivo:
    modelo_redeNeural_base = pickle.load(arquivo)

## Construção da rede neural + resultados - word2vec com cbow

A rede neural abaixo foi desenvolvida com o tutorial mencionado no ínicio do notebook, que, nesse caso o dataframe utilizado é a variável do Word2Vec com o CBoW. 

### Dataframe do word2vec com o cbow


In [68]:
df_vec

,Frase,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,Vetor6,Vetor7,Vetor8,Vetor9,...,Vetor42,Vetor43,Vetor44,Vetor45,Vetor46,Vetor47,Vetor48,Vetor49,Vetor50,sentimento
0,"[alvarez, marsal, estar, conosco, sportainmet,...",0.214823,-0.123036,0.220707,-0.079739,-0.019442,0.202300,0.073742,0.042477,0.057268,...,0.012086,-0.096122,0.148153,0.092629,-0.027504,0.149860,-0.029617,0.032065,0.190372,1
1,"[btgpactual, with, make_repost, entendar, impa...",0.220860,-0.123713,0.205813,-0.063700,-0.010433,0.195687,0.072226,0.026175,0.058045,...,0.006447,-0.076174,0.157446,0.085306,-0.001963,0.156306,-0.019182,0.035594,0.192635,1
2,"[minuto, touro, ouro]",0.265227,-0.068285,0.152235,-0.044329,-0.102729,0.141353,0.092800,0.113174,0.015783,...,0.078032,-0.202677,0.155750,0.062291,0.007038,0.134573,0.014635,0.034189,0.345674,2
3,"[querer, saber, banking, administrar]",0.189241,-0.169407,0.281181,-0.093912,-0.001968,0.181633,0.080923,0.055674,0.103454,...,0.092738,-0.160601,0.179499,0.091803,-0.063762,0.171190,-0.043618,0.030563,0.196250,2
4,"[início, guerra, rússia, ucrânia, velho, apare...",0.219838,-0.138311,0.229412,-0.057420,-0.014510,0.212875,0.058865,0.068096,0.060398,...,0.040836,-0.112662,0.170401,0.094290,-0.046000,0.168181,-0.018029,0.027667,0.180134,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8035,"[excelente, explicação]",0.190917,-0.133475,0.241675,-0.053180,0.067256,0.201138,0.034109,-0.078718,-0.066131,...,-0.082151,0.016113,0.154861,0.068700,-0.004302,0.079717,-0.028388,-0.017448,0.188785,2
8036,"[atendar, telefone, amor, deus]",0.188641,-0.119377,0.199339,-0.105448,0.023176,0.178837,0.069476,-0.004494,0.034710,...,0.034035,-0.126673,0.165176,0.080313,-0.024160,0.118848,-0.003502,0.087053,0.215656,2
8037,"[grande, fiis, mercado, não, selecionar, princ...",0.219437,-0.141124,0.217854,-0.065569,-0.020409,0.205403,0.061334,0.041665,0.063050,...,0.031438,-0.099000,0.176588,0.092546,-0.046104,0.145734,-0.019857,0.029422,0.183504,2
8038,"[erro, financeiro, eliminar, parar, ostentar, ...",0.218153,-0.146420,0.245234,-0.056113,-0.036187,0.224789,0.074320,0.045111,0.040841,...,0.035260,-0.126793,0.171115,0.095479,-0.033473,0.158747,-0.006721,0.029326,0.198914,1


### Separação treino e teste

In [69]:
x, y = df_vec["Frase"], df_vec["sentimento"]

labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)

words = ["o", "ao", 'aos', 'os', 'a', 'as', 'e', 'um', 'uma', 
        'ele', 'ela', 'eles', 'elas', 'do', 'da', 'dos', 'das', 
        'de', 'no', 'na', 'nos', 'nas', 'pelo', 'pela', 'pelos', 
        'pelas', 'num', 'numa', 'nuns', 'numas', 'dum', 'duma', 
        'duns', 'dumas']

x_filter = []

for title in x:
  for word in words:
    title = title.replace(word, '')
  x_filter.append(title)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_filter)

vocab = len(tokenizer.word_docs) + 1

x_filter = tokenizer.texts_to_sequences(x_filter)

max_length = max([len(z) for z in x_filter])
x_filter = pad_sequences(x_filter, maxlen=max_length, padding='post')

x_train, x_test, y_train, y_test = train_test_split(x_filter, y, test_size=0.33)

print("Tamanho de x:", len(x_filter))
print("Tamanho de y:", len(y))


Tamanho de x: 8040
Tamanho de y: 8040


### Criação do modelo

In [70]:
model = Sequential()
model.add(Embedding(input_dim=vocab, output_dim=80, input_length=max_length, trainable = True))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.3))
model.add(Dense(units = 7, activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

mc = ModelCheckpoint('weight.best.hdf5', monitor='val_acc', save_best_only=True, mode='max')

model.fit(x_train, y_train, validation_data = (x_test, y_test), batch_size = 32, epochs = 5, callbacks = [mc])

print(model.evaluate(x_test, y_test))

Epoch 1/5
168/169 [============================>.] - ETA: 0s - loss: 1.4739 - accuracy: 0.4488

169/169 [==============================] - 4s 20ms/step - loss: 1.4735 - accuracy: 0.4484 - val_loss: 1.0951 - val_accuracy: 0.4797
Epoch 2/5
169/169 [==============================] - ETA: 0s - loss: 1.0064 - accuracy: 0.5488

169/169 [==============================] - 3s 17ms/step - loss: 1.0064 - accuracy: 0.5488 - val_loss: 0.9402 - val_accuracy: 0.5912
Epoch 3/5
169/169 [==============================] - ETA: 0s - loss: 0.8677 - accuracy: 0.6396

169/169 [==============================] - 3s 17ms/step - loss: 0.8677 - accuracy: 0.6396 - val_loss: 0.8578 - val_accuracy: 0.6368
Epoch 4/5
169/169 [==============================] - ETA: 0s - loss: 0.7578 - accuracy: 0.7159

169/169 [==============================] - 3s 19ms/step - loss: 0.7578 - accuracy: 0.7159 - val_loss: 0.8043 - val_accuracy: 0.6613
Epoch 5/5
167/169 [============================>.] - ETA: 0s - loss: 0.6736 - accuracy: 0.7627

83/83 [==============================] - 0s 2ms/step - loss: 0.7704 - accuracy: 0.6718
[0.7703697681427002, 0.6718161106109619]


In [71]:
# Avaliação do modelo
loss, accuracy = model.evaluate(x_test, y_test)
print('Loss:', loss)
print('Accuracy:', accuracy)

83/83 [==============================] - 0s 2ms/step - loss: 0.7704 - accuracy: 0.6718
Loss: 0.7703697681427002
Accuracy: 0.6718161106109619


### Exportação com a biblioteca pickle

In [72]:
with open('redeNeural_word2vec_cbow.pkl', 'wb') as arquivo:
    pickle.dump(model, arquivo)
with open('redeNeural_word2vec_cbow.pkl', 'rb') as arquivo:
    modelo_redeNeural_word2vec_cbow = pickle.load(arquivo)

## Construção da rede neural + resultados - word2vec com embedding layer

A rede neural abaixo foi desenvolvida com o tutorial mencionado no ínicio do notebook, que, nesse caso o dataframe utilizado é a variável do Word2Vec com Embedding Layer.

### Dataframe do word2vec com embedding layer

In [73]:
df_word2vec

,texto_tratado,sentimentoNumerico,Vetor0,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,Vetor6,Vetor7,...,Vetor90,Vetor91,Vetor92,Vetor93,Vetor94,Vetor95,Vetor96,Vetor97,Vetor98,Vetor99
0,"[alvarez, marsal, estar, conosco, sportainmet,...",1,0.297414,0.006346,0.135038,0.061838,-0.472301,0.233506,0.117101,-0.425061,...,-0.529049,-0.093984,-0.277712,0.600598,-0.224913,0.077593,-0.300101,0.015710,-0.027120,-0.124745
1,"[btgpactual, with, make_repost, entendar, impa...",1,0.281173,0.013540,0.112198,0.077889,-0.447345,0.228404,0.106523,-0.412592,...,-0.493123,-0.097028,-0.298142,0.570242,-0.218310,0.051990,-0.291846,0.035265,-0.034986,-0.114778
2,"[minuto, touro, ouro]",2,0.288293,0.077489,0.083979,-0.029461,-0.449480,0.319432,0.230896,-0.553132,...,-0.526300,-0.094131,-0.340655,0.586246,-0.188992,0.181223,-0.175162,-0.040045,0.063862,-0.208368
3,"[querer, saber, banking, administrar]",2,0.268780,0.011891,0.103685,0.073322,-0.418163,0.195450,0.059639,-0.359379,...,-0.505033,-0.096492,-0.281548,0.480524,-0.226503,0.106688,-0.250742,-0.008421,-0.057605,-0.068400
4,"[início, guerra, rússia, ucrânia, velho, apare...",0,0.273418,0.049957,0.097502,0.070372,-0.475558,0.255420,0.138153,-0.459308,...,-0.529097,-0.081015,-0.279064,0.594373,-0.248605,0.046594,-0.286832,0.044750,-0.021001,-0.106887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8035,"[excelente, explicação]",2,0.233112,0.151018,0.135960,0.022766,-0.479874,0.150638,0.130819,-0.328104,...,-0.456086,-0.174277,-0.319233,0.522279,-0.304542,-0.031399,-0.251650,0.073567,-0.034155,-0.137709
8036,"[atendar, telefone, amor, deus]",2,0.307020,0.050195,0.037812,0.063392,-0.421072,0.260814,0.164793,-0.466451,...,-0.526184,-0.092594,-0.331005,0.592767,-0.261083,0.003219,-0.327767,0.041722,-0.047595,-0.152317
8037,"[grande, fiis, mercado, não, selecionar, princ...",2,0.284146,0.047818,0.108436,0.064484,-0.500483,0.272070,0.147573,-0.475484,...,-0.501176,-0.081624,-0.290470,0.594903,-0.255342,0.034344,-0.309130,0.058736,-0.004545,-0.090214
8038,"[erro, financeiro, eliminar, parar, ostentar, ...",1,0.281530,0.070156,0.116428,0.075205,-0.506874,0.290723,0.140936,-0.486017,...,-0.524735,-0.087025,-0.276499,0.614986,-0.221881,0.053634,-0.317234,0.051381,-0.020959,-0.114861


### Separação treino e teste


In [74]:
x, y = df_word2vec["texto_tratado"], df_word2vec["sentimentoNumerico"]

labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)

words = ["o", "ao", 'aos', 'os', 'a', 'as', 'e', 'um', 'uma', 
        'ele', 'ela', 'eles', 'elas', 'do', 'da', 'dos', 'das', 
        'de', 'no', 'na', 'nos', 'nas', 'pelo', 'pela', 'pelos', 
        'pelas', 'num', 'numa', 'nuns', 'numas', 'dum', 'duma', 
        'duns', 'dumas']

x_filter = []

for title in x:
  for word in words:
    title = title.replace(word, '')
  x_filter.append(title)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_filter)

vocab = len(tokenizer.word_docs) + 1

x_filter = tokenizer.texts_to_sequences(x_filter)

max_length = max([len(z) for z in x_filter])
x_filter = pad_sequences(x_filter, maxlen=max_length, padding='post')

x_train, x_test, y_train, y_test = train_test_split(x_filter, y, test_size=0.33)

print("Tamanho de x:", len(x_filter))
print("Tamanho de y:", len(y))


Tamanho de x: 8040
Tamanho de y: 8040


### Criação do modelo

In [75]:
model = Sequential()
model.add(Embedding(input_dim=vocab, output_dim=80, input_length=max_length, trainable = True))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.3))
model.add(Dense(units = 7, activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

mc = ModelCheckpoint('weight.best.hdf5', monitor='val_acc', save_best_only=True, mode='max')

model.fit(x_train, y_train, validation_data = (x_test, y_test), batch_size = 32, epochs = 5, callbacks = [mc])

print(model.evaluate(x_test, y_test))

Epoch 1/5
169/169 [==============================] - ETA: 0s - loss: 1.4909 - accuracy: 0.4009

169/169 [==============================] - 4s 18ms/step - loss: 1.4909 - accuracy: 0.4009 - val_loss: 1.1193 - val_accuracy: 0.5008
Epoch 2/5
166/169 [============================>.] - ETA: 0s - loss: 1.0194 - accuracy: 0.5292

169/169 [==============================] - 3s 17ms/step - loss: 1.0182 - accuracy: 0.5295 - val_loss: 0.9526 - val_accuracy: 0.6179
Epoch 3/5
169/169 [==============================] - ETA: 0s - loss: 0.8624 - accuracy: 0.6697

169/169 [==============================] - 3s 20ms/step - loss: 0.8624 - accuracy: 0.6697 - val_loss: 0.8577 - val_accuracy: 0.6522
Epoch 4/5
166/169 [============================>.] - ETA: 0s - loss: 0.7466 - accuracy: 0.7374

169/169 [==============================] - 4s 23ms/step - loss: 0.7473 - accuracy: 0.7373 - val_loss: 0.7945 - val_accuracy: 0.6760
Epoch 5/5
166/169 [============================>.] - ETA: 0s - loss: 0.6483 - accuracy: 0.7839

83/83 [==============================] - 0s 2ms/step - loss: 0.7637 - accuracy: 0.6809
[0.7636685371398926, 0.6808590888977051]


In [76]:
# Avaliação do modelo
loss, accuracy = model.evaluate(x_test, y_test)
print('Loss:', loss)
print('Accuracy:', accuracy)

83/83 [==============================] - 0s 2ms/step - loss: 0.7637 - accuracy: 0.6809
Loss: 0.7636685371398926
Accuracy: 0.6808590888977051


### Exportação com a biblioteca pickle 

In [77]:
with open('redeNeural_word2vec_embedding.pkl', 'wb') as arquivo:
    pickle.dump(model, arquivo)
with open('redeNeural_word2vec_embedding.pkl', 'rb') as arquivo:
    modelo_redeNeural_word2vec_embedding = pickle.load(arquivo)